In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.offline as pyo
import plotly.graph_objs as go
# import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
mob_money = pd.read_csv('training.csv')

In [3]:
# changing index cols with rename()
mob_money = mob_money.rename(columns = {"Q1": "Age",
                                  "Q2": "Gender",
                                  "Q3": "Marital Status",
                                  "Q4": "Highest Qualification",
                                  "Q5": "Living Conditions",
                                  "Q6": "Land Ownership",
                                  "Q7": "Cellphone Ownership",
                                  "Q8_1": "Salaries/Wages",
                                  "Q8_2": "Trading",
                                  "Q8_3": "Service Provider",
                                  "Q8_4": "Piece Jobs",
                                  "Q8_5": "Rental Income",
                                  "Q8_6": "Investments",
                                  "Q8_7": "Pension",
                                  "Q8_8": "Social Welfare",
                                  "Q8_9": "Rely on Someone Else",
                                  "Q8_10": "No Money",
                                  "Q8_11": "Other",
                                  "Q9": "Employer",
                                  "Q10": "Trading Products",
                                  "Q11": "Services Provided",
                                  "Q12": "Sent Money(12months)",
                                  "Q13": "Last Sent Money",
                                  "Q14": "Received Money(12months)",
                                  "Q15": "Last Received Money",
                                  "Q16": "Mobile Money for GnS (12months)",
                                  "Q17": "Mobile Money for Bills (12months)",
                                  "Q18": "Literacy in Kiswhahili",
                                  "Q19": "Literacy in English"})

# changing columns using .columns()
mob_money.columns = ["ID","Age", "Gender","Marital Status","Highest Qualification","Living Conditions",
                    "Land Ownership", "Cellphone Ownership", "Salaries/Wages","Trading", "Service Provider",
                    "Piece Jobs", "Rental Income","Investments", "Pension","Social Welfare",
                    "Rely on Someone Else", "No Money","Other","Employer","Trading Products",
                    "Services Provided","Sent Money(12months)","Last Sent Money","Received Money(12months)",
                    "Last Received Money","Mobile Money for GnS (12months)", "Mobile Money for Bills (12months)",
                    "Literacy in Kiswhahili","Literacy in English","Latitude","Longitude","mobile_money","savings",
                    "borrowing","insurance","mobile_money_classification"]

In [4]:
# changing the values in columns from intergers to representative strings
mob_money['Gender']= ['Female' if each==2 else 'Male' for each in mob_money['Gender']]
mob_money['Land Ownership']= ['No' if each==2 else 'Yes' for each in mob_money['Land Ownership']]
mob_money['Sent Money(12months)']= ['No' if each==2 else 'Yes' for each in mob_money['Sent Money(12months)']]
mob_money['Received Money(12months)']= ['No' if each==2 else 'Yes' for each in mob_money['Received Money(12months)']]
mob_money['Last Sent Money'] = mob_money['Last Sent Money'].map({-1: "N/A", 4: "90Days", 2: "7Days", 5: "90+Days", 3: "30Days", 6: "6Mnth+", 1: "Y/Today"})
mob_money['Last Received Money'] = mob_money['Last Received Money'].map({-1: "N/A", 5: "90+Days", 1: "Y/Today", 3: "30Days", 4: "90Days", 6: "6Mnth+", 2: "7Days"})
mob_money['Mobile Money for GnS (12months)'] = mob_money['Mobile Money for GnS (12months)'].map({-1: "N/A", 1: "Never", 4: "Monthly", 5: "LessMnthly", 3: "Weekly", 2: "Daily"})
mob_money['Mobile Money for Bills (12months)'] = mob_money['Mobile Money for Bills (12months)'].map({-1: "N/A", 4: "Monthly", 1: "Never", 2: "Daily", 5: "LessMnthly", 3: "Weekly"})
mob_money['mobile_money']= ['No' if each==0 else 'Yes' for each in mob_money['mobile_money']]
mob_money['mobile_money_classification'] = mob_money['mobile_money_classification'].map({0: "None", 3: "MM_Plus", 2: "MM_Only", 1: "Other_Only"})
mob_money['Marital Status'] = mob_money['Marital Status'].map({3: "Widowed", 1: "Married", 4: "Single", 2: "Divorced"})

In [5]:
females = mob_money[mob_money['Gender'] == 'Female']
males = mob_money[mob_money['Gender'] == 'Male']
f_none = len(females[females.mobile_money_classification == "None"])
f_plus = len(females[females.mobile_money_classification == "MM_Plus"])
f_only = len(females[females.mobile_money_classification == "MM_Only"])
f_other = len(females[females.mobile_money_classification == "Other_Only"])

m_none = len(males[males.mobile_money_classification == "None"])
m_plus = len(males[males.mobile_money_classification == "MM_Plus"])
m_only = len(males[males.mobile_money_classification == "MM_Only"])
m_other = len(males[males.mobile_money_classification == "Other_Only"])

In [6]:
# creating an incomes dataframe to compare incomes with mobile money usage
incomes = mob_money.loc[:, 'Salaries/Wages':'Other']
incomes['mobile_money'] = mob_money['mobile_money']
incomes['Sent Money(12months)'] = mob_money['Sent Money(12months)']
incomes['Received Money(12months)'] = mob_money['Received Money(12months)']
incomes = pd.melt(incomes, id_vars = ['mobile_money', 'Sent Money(12months)', 'Received Money(12months)'], var_name='Income_Type') 
incomes = incomes[incomes['value'] == 1]
incomes.Income_Type.value_counts()

Trading                 4470
Piece Jobs              2393
Rely on Someone Else    1112
Salaries/Wages           441
Service Provider         412
No Money                 410
Social Welfare           131
Rental Income             67
Investments               34
Pension                   34
Other                      9
Name: Income_Type, dtype: int64

In [9]:
chart1 = pd.DataFrame(mob_money["mobile_money_classification"].value_counts()).reset_index()
chart1

,index,mobile_money_classification
0,MM_Plus,3126
1,Other_Only,1778
2,None,1386
3,MM_Only,804


In [14]:
chart2 = pd.DataFrame(mob_money['Mobile Money for GnS (12months)'].value_counts()).reset_index()
chart2

,index,Mobile Money for GnS (12months)
0,Never,5089
1,LessMnthly,1137
2,Monthly,543
3,Weekly,248
4,Daily,77


In [15]:
chart3 = pd.DataFrame(incomes['Income_Type'].value_counts()).reset_index()
chart3

,index,Income_Type
0,Trading,4470
1,Piece Jobs,2393
2,Rely on Someone Else,1112
3,Salaries/Wages,441
4,Service Provider,412
5,No Money,410
6,Social Welfare,131
7,Rental Income,67
8,Investments,34
9,Pension,34


In [8]:
females = mob_money[mob_money['Gender'] == 'Female']
males = mob_money[mob_money['Gender'] == 'Male']